In [ ]:
!pip install gensim
!pip install jieba
!pip install underthesea

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/LDS0_K271_NGUYENMINHQUANG/Project3/'

/content/gdrive/MyDrive/LDS0_K271_NGUYENMINHQUANG/Project3


# Đọc dữ liệu

In [ ]:
products = pd.read_csv('Products.csv')
reviews = pd.read_csv('Reviews.csv', lineterminator='\n')

In [ ]:
products.head(2)

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4373 entries, 0 to 4372
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      4373 non-null   int64  
 1   name         4373 non-null   object 
 2   description  4370 non-null   object 
 3   rating       4373 non-null   float64
 4   price        4373 non-null   int64  
 5   list_price   4373 non-null   int64  
 6   brand        4373 non-null   object 
 7   group        4373 non-null   object 
 8   url          4373 non-null   object 
 9   image        4373 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 341.8+ KB


# Underthesea

word_tokenize, pos_tag, sent_tokenize

    link: https://github.com/undertheseanlp/underthesea

In [ ]:
products = products[products['name'].notnull()]

In [ ]:
products['name_description'] = products.name + products.description

In [ ]:
products = products[products['name_description'].notnull()]

In [ ]:
products['name_description_pre'] = products['name_description'].apply(lambda x: word_tokenize(x, format='text'))

In [ ]:
type(products)

pandas.core.frame.DataFrame

In [ ]:
products.shape

(4370, 12)

In [ ]:
products = products.reset_index()

# Gensim

- https://pypi.org/project/gensim/ -Là một thư viện Python chuyên xác định sự tương tự về ngữ nghĩa giữa hai tài liệu thông qua mô hình không gian vector và bộ công cụ mô hình hóa chủ đề.
- Có thể xử lý kho dữ liệu văn bản lớn với sự trợ giúp của việc truyền dữ liệu hiệu quả và các thuật toán tăng cường
- Tốc độ xử lý và tối ưu hóa việc sử dụng bộ nhớ tốt
- Tuy nhiên, Gensim có ít tùy chọn tùy biến cho các function

#### Tham khảo:
- link https://www.tutorialspoint.com/gensim/index.htm
- link https://www.machinelearningplus.com/nlp/gensim-tutorial/


In [ ]:
# Tokenize(split) the sentences into words
intro_products = [[text for text in x.split()] for x in products.name_description_pre]

In [ ]:
len(intro_products)

4370

In [ ]:
intro_products[:1]

[['Tai_nghe',
  'Bluetooth_Inpods',
  '12',
  '-',
  'Cảm_biến',
  'vân',
  'tay',
  ',',
  'chống',
  'nước',
  ',',
  'màu_sắc',
  'đa_dạng',
  '-',
  '5',
  'màu_sắc',
  'lựa',
  'chọnTHÔNG',
  'TIN',
  'CHI_TIẾT',
  'Dung_lượng',
  'pin',
  '300',
  'Thời_gian',
  'pin',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  '-',
  'Thời_gian',
  'sạc',
  'đầy',
  'chỉ',
  'khoảng',
  '60',
  'p',
  '-',
  'Thời_gian',
  'chờ',
  'lên',
  'tới',
  '140',
  'giờ',
  'Bluetooth',
  '5',
  'Thương_hiệu',
  'OEM',
  'Xuất_xứ',
  'thương_hiệu',
  'Trung_Quốc',
  'Độ',
  'nhạy_cảm_biến',
  'vân',
  'tay',
  'Model',
  'i12',
  'Loại',
  'Jack',
  'cắm',
  'USB_Cable',
  'Trọng_lượng',
  '300',
  'g',
  'Thời_gian',
  'sử_dụng',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  'SKU',
  '4096608751631',
  'MÔ_TẢ',
  'SẢN_PHẨM',
  'INPOD',
  '12',
  'là',
  'phiên_bản',
  'nâng_cấp',
  'mới',
  'nhất',
 

In [ ]:
# Stop words
STOP_WORD_FILE = 'vietnamese-stopwords.txt'

In [ ]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

In [ ]:
# remove some special elements in texts
intro_products_re = [[re.sub('[0-9]+','', e) for e in text] 
                        for text in intro_products] # số
intro_products_re = [[t.lower() 
                        for t in text if not t in [
                          '', ' ', ',', '.', '...', '-',':', 
                          ';', '?', '%', '(', ')', '+', '/']] 
                            for text in  intro_products_re] # ký tự đặc biệt
intro_products_re = [[t for t in text if not t in stop_words]
                        for text in intro_products_re] # stopword

In [ ]:
intro_products_re[:1]

[['tai_nghe',
  'bluetooth_inpods',
  'cảm_biến',
  'vân',
  'chống',
  'màu_sắc',
  'đa_dạng',
  'màu_sắc',
  'lựa',
  'chọnthông',
  'chi_tiết',
  'dung_lượng',
  'pin',
  'pin',
  'nhạc',
  'liên_tục',
  'h',
  'sạc',
  'p',
  'chờ',
  'bluetooth',
  'thương_hiệu',
  'oem',
  'xuất_xứ',
  'thương_hiệu',
  'trung_quốc',
  'độ',
  'nhạy_cảm_biến',
  'vân',
  'model',
  'jack',
  'cắm',
  'usb_cable',
  'trọng_lượng',
  'g',
  'nhạc',
  'liên_tục',
  'h',
  'sku',
  'mô_tả',
  'sản_phẩm',
  'inpod',
  'phiên_bản',
  'nâng_cấp',
  'tai_nghe',
  'bluetooth',
  'thiết_kế',
  'tỉ_lệ',
  'chuẩn',
  'tai',
  'airpod',
  'hãng',
  'lược_bỏ',
  'nút',
  'bấm',
  'thân',
  'tai',
  'thay',
  'nút',
  'cảm_ứng',
  'dễ_dàng',
  'thuận_tiện',
  'thao_tác',
  'nhạc',
  'dễ_dàng',
  'chạm',
  'bluetooth',
  'kết_nối',
  'vô_cùng',
  'ổn_định',
  'bluetooth',
  'kết_nối',
  'vô_cùng',
  'ổn_định',
  'tai',
  'kết_nối',
  'dock',
  'sạc',
  'chất',
  'âm',
  'thời_lượng',
  'pin',
  'cải_thiện',
  'tố

In [ ]:
# Obtain the number of features based on dictionary:
## Use corpora.Dictionary
dictionary = corpora.Dictionary(intro_products_re)

In [ ]:
# List od features in dictionary
dictionary.token2id

{'airpod': 0,
 'apple': 1,
 'bao_gồm': 2,
 'bluetooth': 3,
 'bluetooth_inpods': 4,
 'bấm': 5,
 'chi_phí': 6,
 'chi_tiết': 7,
 'chuẩn': 8,
 'chạm': 9,
 'chất': 10,
 'chọnthông': 11,
 'chống': 12,
 'chờ': 13,
 'cải_thiện': 14,
 'cảm_biến': 15,
 'cảm_ứng': 16,
 'cắm': 17,
 'cồng_kềnh': 18,
 'dock': 19,
 'dung_lượng': 20,
 'dễ_dàng': 21,
 'g': 22,
 'giao': 23,
 'giá': 24,
 'h': 25,
 'hiện_hành': 26,
 'huawei': 27,
 'hàng': 28,
 'hãng': 29,
 'inpod': 30,
 'jack': 31,
 'kết_nối': 32,
 'lenovo': 33,
 'liên_tục': 34,
 'luật': 35,
 'lược_bỏ': 36,
 'lựa': 37,
 'model': 38,
 'màu_sắc': 39,
 'mô_tả': 40,
 'nhạc': 41,
 'nhạy_cảm_biến': 42,
 'nâng_cấp': 43,
 'nút': 44,
 'oem': 45,
 'oppo': 46,
 'p': 47,
 'phiên_bản': 48,
 'phát_sinh': 49,
 'phí': 50,
 'phương_thức': 51,
 'phụ_phí': 52,
 'pin': 53,
 'samsung': 54,
 'sku': 55,
 'sạc': 56,
 'sản_phẩm': 57,
 'tablet': 58,
 'tai': 59,
 'tai_nghe': 60,
 'thao_tác': 61,
 'thay': 62,
 'thiết_bị': 63,
 'thiết_kế': 64,
 'thuận_tiện': 65,
 'thuế': 66,
 'thân':

In [ ]:
len(dictionary.token2id)

37210

In [ ]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)
feature_cnt

37210

In [ ]:
# Obtain corpus based on dictionary (dense matrix)
corpus = [dictionary.doc2bow(text) for text in intro_products_re]

In [ ]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 5),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 2),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 2),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 3),
 (26, 1),
 (27, 1),
 (28, 2),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 3),
 (33, 1),
 (34, 3),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 2),
 (40, 1),
 (41, 4),
 (42, 1),
 (43, 1),
 (44, 2),
 (45, 1),
 (46, 1),
 (47, 2),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 4),
 (54, 1),
 (55, 1),
 (56, 5),
 (57, 3),
 (58, 1),
 (59, 3),
 (60, 3),
 (61, 1),
 (62, 1),
 (63, 2),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 2),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 2),
 (80, 1),
 (81, 2),
 (82, 2),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 2)]

In [ ]:
# Use TF-IDF to process corpous, obtaning index
tfidf = models.TfidfModel(corpus)
# Tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=feature_cnt)

In [ ]:
# When user choose one product
# Giả sử là chọn sản phẩm đầu tiên để xem, index=0
product_ID = 10001355
product = products[products.item_id == product_ID].head(1)

In [ ]:
type(product['name_description_pre'])

pandas.core.series.Series

In [ ]:
product[['index', 'item_id', 'name_description_pre']]

,index,item_id,name_description_pre
2658,2660,10001355,RAM Laptop_Hynix 4GB DDR4 2400MH z SODIMM - Hà...


In [ ]:
# Sản phẩm đang xem
name_description_pre = product['name_description_pre'].to_string(index=False)

In [ ]:
name_description_pre

' RAM Laptop_Hynix 4GB DDR4 2400MH z SODIMM - Hà...'

In [ ]:
# Dề xuất các sản phẩm đang xem
def recommender(view_product, dictionary, tfidf, index):
  # Convert search words into Sparse vectors
  view_product = view_product.lower().split()
  # Convert search words into Sparse Vectors
  kw_vector = dictionary.doc2bow(view_product)
  print("View product's vector:")
  print(kw_vector)
  # Similarity calculation
  sim  = index[tfidf[kw_vector]]

  # print result
  list_id = []
  list_score = []
  for i in range(len(sim)):
    list_id.append(i)
    list_score.append(sim[i])
  
  df_result = pd.DataFrame({'id':list_id, 'score':list_score})
 
  # 5 highest scores
  five_highest_score = df_result.sort_values(by='score', 
                                             ascending=False).head(6)
  print('Five highest score:')
  print(five_highest_score)
  print('Ids to list:')
  idTolist = list(five_highest_score['id'])
  print(idTolist)

  products_find = products[products.index.isin(idTolist)]
  results = products_find[['index', 'item_id', 'name']]
  results = pd.concat([results, five_highest_score], 
                      axis=1).sort_values(by='score', 
                                          ascending=False)
  return results

In [ ]:
results = recommender(name_description_pre, dictionary, tfidf, index)

View product's vector:
[(306, 1), (318, 1), (19609, 1), (21532, 1)]
Five highest score:
        id     score
2658  2658  0.442807
2505  2505  0.413952
2473  2473  0.390643
2662  2662  0.376236
2314  2314  0.298801
2413  2413  0.279375
Ids to list:
[2658, 2505, 2473, 2662, 2314, 2413]


In [ ]:
# Recommender 5 similarity products for the selected product
# Check and remove the selected product from the results
results = results[results.item_id != product_ID]
results

,index,item_id,name,id,score
2505,2507,10001357,RAM Laptop Hynix 8GB DDR4 2400MHz SODIMM - Hàn...,2505,0.413952
2473,2474,10001377,RAM Laptop Hynix 8GB DDR4 2666MHz SODIMM - Hàn...,2473,0.390643
2662,2664,10001334,RAM Laptop Hynix 8GB DDR4 2133MHz SODIMM - Hàn...,2662,0.376236
2314,2315,10001369,RAM Laptop Samsung 8GB DDR4 2666MHz SODIMM - H...,2314,0.298801
2413,2414,10001373,RAM Laptop Samsung 16GB DDR4 2666MHz SODIMM - ...,2413,0.279375


In [ ]:
# Lưu lại các tham số dictionary, tfidf, index để có thể đọc ở bất cứ đâu
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [ ]:
type(tfidf)

gensim.models.tfidfmodel.TfidfModel

In [ ]:
type(index)

gensim.similarities.docsim.SparseMatrixSimilarity